In [1]:
# %matplotlib notebook
# import geopandas as gpd
# import pandas as pd
# import matplotlib.pyplot as plt
# from tqdm import tqdm
# import contextily as cx
import xml.etree.ElementTree as ET
# from shapely import wkt
import random
import sumolib.net as sumonet
import pandas as pd
from tqdm import tqdm

# 1- rou.xml adjustment
### remove selected starting edges from od2trips results
First, make sure you added vehicle type to start of that file

In [14]:
make_north_more_congested = True  # it is true when we want to have a more congested network to use in hte segmentation  
file_name = 'alaskanway_od2trips_psrc_vehicle_trips_converted_taz_calibrated_V4.rou.xml' # V4 is used for segmentation and mfd generation

# file_name = 'alaskanway_od2trips_psrc_vehicle_trips_converted_taz_calibrated_V3.rou.xml'  # V3 is used for calibration

tree = ET.parse(file_name)
root = tree.getroot()

In [15]:
from_tazs_fix_dep_lane = [5004, 5002]
highways_and_side_ptaz_origins = [5000, 5002, 5004, 5008, 5010, 7000, 7001, 7002, 7003, 7004, 7005]


for trip in (root.findall('trip')):
    otaz = int(trip.get('fromTaz'))
    if otaz in from_tazs_fix_dep_lane:
        if random.random() <0.7:  # we assume that there is a 30% probability that cars enter the network with 'free' departLane
            trip.set('departLane', 'best')
    
    if otaz not in highways_and_side_ptaz_origins:
        if random.random() < 0.5:
            trip.set('departLane', 'first')
            trip.set('departSpeed', 'random')  
            # we prefer to use 0 but 0 might cause delay in insertion. And 'random' is better than 'max'

#### or maybe 20% best and %20% right for all (exept for highways)

In [16]:
def handle_north_trip(trip):
    olink = trip.get('from')
    dlink = trip.get('to')
    otaz = int(trip.get('fromTaz'))
    dtaz = int(trip.get('toTaz'))
#     if int(trip.get('id'))==2629:
#     print(int(trip.get('id')),olink, dlink, otaz, dtaz)
    
    if dtaz in [7002, 7003]:
        del trip.attrib['to']
    elif 'pseudoMercer' in dlink:
        del trip.attrib['to']
    else:
        del trip.attrib['toTaz']
    
    
    if otaz in [7002, 7003]:
        del trip.attrib['from']
    elif 'pseudoMercer' in olink:
        del trip.attrib['from']
    else:
        del trip.attrib['fromTaz']

In [17]:
sumo_east = [426, 427, 428, 429, 547, 548, 549, 550, 551, 552, 553, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 604, 605, 606, 607, 608, 609, 615, 627, 636]

sumo_south = [526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 577, 578, 579, 580, 581, 582, 583, 584, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 627, 628, 629, 630, 631, 632, 634, 635, 636, 650, 651, 652, 653]

In [18]:
north_TAZs = [442, 440, 438, 436, 434, 432, 430]  # see limitations in the doc

from_tazs= [636, 5000, 5004, 5010, 6001, 7000, 7001, 7002, 7003, 7004, 7005] + north_TAZs  # all??
to_tazs = [636, 5005, 5011, 6001, 7000, 7001, 7002, 7003, 7004, 7005] + north_TAZs

def trip_btw_northTAZs_and_upper_pTAZs(otaz, dtaz):
    if (otaz in north_TAZs) and (dtaz in north_TAZs):
        return True
    if (otaz in north_TAZs) and (dtaz in [7002, 7003]):
        return True
    if (otaz in [7002, 7003]) and (dtaz in north_TAZs):
        return True
    return False

def trip_btw_northTAZs_and_sumo_northwest(otaz, dtaz):
    if make_north_more_congested:
        if (otaz in north_TAZs) and (dtaz not in sumo_east) and (dtaz not in sumo_south):
            return True
        if (otaz not in sumo_east) and (otaz not in sumo_south) and (dtaz in north_TAZs):
            return True
    return False


for trip in (root.findall('trip')):
    otaz = int(trip.get('fromTaz'))
    dtaz = int(trip.get('toTaz'))  
    olink = trip.get('from')
    dlink = trip.get('to')

    if otaz in from_tazs and dtaz in to_tazs:
        if trip_btw_northTAZs_and_upper_pTAZs(otaz, dtaz):
            handle_north_trip(trip)
            # delete this condition and run the simulation to see the mess we had! 
            # we don't want trips only use pseudoMercer links
        else: 
            del trip.attrib['to']
            del trip.attrib['from']
        
    elif otaz in from_tazs:
        if trip_btw_northTAZs_and_sumo_northwest(otaz, dtaz) and ('pseudoMercer' not in olink):
            del trip.attrib['fromTaz']
        else:
            del trip.attrib['from']  # we just keep fromTaz
        del trip.attrib['toTaz']  
        # if we have toTaz, SUMO will run a TAZ to TAZ route assignment. I think it doesn't happen with od2trips results,
        # but when we 
        
    elif dtaz in to_tazs:  
        if trip_btw_northTAZs_and_sumo_northwest(otaz, dtaz) and ('pseudoMercer' not in dlink):
            del trip.attrib['toTaz']
        else:
            del trip.attrib['to']  # we just keep toTaz
        del trip.attrib['fromTaz'] # if we have toTaz, SUMO will run a TAZ to TAZ route assignment
        
    else:
        del trip.attrib['toTaz']
        del trip.attrib['fromTaz']

In [19]:
tree.write('adjusted_'+file_name, encoding='utf-8', xml_declaration=True)


# 2-Changing weights in TAZs

In [9]:
net = sumonet.readNet('Seattle_road_network.net.xml')
edges = net.getEdges()
edge_p_dict = {}
for edge in edges:
    ID = edge.getID()
    p = edge.getPriority()
    edge_p_dict[ID] = p

In [11]:
edges_in_TAZs = []

# load sumo tazs
tree = ET.parse('new_Taz_with_pseudo_link.add.xml')
root = tree.getroot()
sumo_taz = []
for taz in root.findall('taz'):
#     if str(taz.get('id'))>5000:
    for src in taz.findall('tazSource'):
        link_id = src.get('id')
        edges_in_TAZs.append(link_id)
        link_priority = edge_p_dict[link_id]
        if src.get('weight')!= '0.00':
            src.set('weight', str(link_priority)+'.00')
    for sink in taz.findall('tazSink'):
        link_id = sink.get('id')
        edges_in_TAZs.append(link_id)
        link_priority = edge_p_dict[link_id]
        if sink.get('weight')!= '0.00':
            sink.set('weight', str(link_priority)+'.00')



In [ ]:
# Save the modified XML back to a file
tree.write('modified_new_Taz_with_pseudo.add.xml', encoding='utf-8', xml_declaration=True)

In [14]:
len(edges)

7384

In [15]:
len(set(edges_in_TAZs))

4031

In [16]:
with open('edges_in_TAZs.txt', 'w') as f:
    for edge in set(edges_in_TAZs):
        f.write(edge+'\n')

# reducing speed of alleys

In [14]:
# load sumo tazs
tree = ET.parse('Seattle_road_network.net.xml')
root = tree.getroot()
# sumo_taz = []
for edge in root.findall('edge'):
    if edge.get('function') in['walkingarea', 'internal', 'crossing']:
        continue
    else:
        if type(edge.get('priority'))==str:
            if int(edge.get('priority'))==2:
                for lane in edge.findall('lane'):
                    if float(lane.get('speed'))==6.71:
                        lane.set('speed', str(4.47))  # 15mph->10mph
                    if float(lane.get('speed'))==5.56:
                        lane.set('speed', str(3.35))  # 12.4mph->7.5mph
        else:
            print(edge.get('id'))


In [15]:
# Save the modified XML back to a file
tree.write('new_Seattle_road_network.net.xml', encoding='utf-8', xml_declaration=True)

# deleting some pedestrians after deleting alleys from network

In [6]:
net = sumonet.readNet('no_alley_Seattle_road_network.net.xml')
edges = net.getEdges()

In [7]:
# load sumo tazs
tree = ET.parse('ped_od_input.rou.xml')
root = tree.getroot()
for person in (root.findall('person')):
    for walk in person.findall('walk'):
        if (walk.get('from') in edges) and (walk.get('to') in edges):
            continue
        else:
            root.remove(person)
            

In [8]:
# Save the modified XML back to a file
tree.write('new_ped_od_input.rou.xml', encoding='utf-8', xml_declaration=True)

# filtering demnad

In [20]:
# total number of trips?
tree = ET.parse('new_psrc_vehicle_trips_converted_taz.rou.xml')
root = tree.getroot()
print(len(root.findall('trip')))

166684


In [18]:
# out of downtown to out of downton trips
tree = ET.parse('new_psrc_vehicle_trips_converted_taz.rou.xml')
root = tree.getroot()
counter = 0
for trip in (root.findall('trip')):
    if (int(trip.get('fromTaz')) >= 5000) and (int(trip.get('toTaz')) >= 5000):
        counter += 1
        continue
    else:
        root.remove(trip)
print(counter)
# Save the modified XML back to a file
tree.write('new_psrc_vehicle_trips_converted_taz_no downtown.rou.xml', encoding='utf-8', xml_declaration=True)          

16950


In [17]:
#  downtown to downton trips
tree = ET.parse('new_psrc_vehicle_trips_converted_taz.rou.xml')
root = tree.getroot()
counter = 0
for trip in (root.findall('trip')):
    if (int(trip.get('fromTaz')) < 5000) and (int(trip.get('toTaz')) < 5000):
        counter+=1
        continue
    else:
        root.remove(trip)
print(counter)
# Save the modified XML back to a file
tree.write('new_psrc_vehicle_trips_converted_taz_just downtown.rou.xml', encoding='utf-8', xml_declaration=True)          

20357


In [33]:
# downtown to/from out of downtown trips
tree = ET.parse('new_psrc_vehicle_trips_converted_taz.rou.xml')
root = tree.getroot()
counter = 0
for trip in (root.findall('trip')):
    if (
        ((int(trip.get('fromTaz')) >= 5000) and (int(trip.get('toTaz')) < 5000)) 
        or ((int(trip.get('fromTaz')) < 5000) and (int(trip.get('toTaz')) >= 5000))
           ):
        counter+=1
        continue
    else:
        root.remove(trip)
print(counter)
# Save the modified XML back to a file
tree.write('new_psrc_vehicle_trips_converted_taz_one end downtown.rou.xml', encoding='utf-8', xml_declaration=True)          

129377


# Reducing demand

In [2]:
# tree = ET.parse('modified_new_psrc_vehicle_trips_converted_taz.rou.xml')
tree = ET.parse('od2trips_new_psrc_vehicle_trips_converted_taz.rou.xml') 
# make sure in this file, rows you want to change have the desired attribute (like 'fromTaz')
root = tree.getroot()

# reduce_taz = [5010, 5008, 5000, 5002]
# reduce_taz = {5010:30, 5002:20, 5000:5, 5008:15}
# reduce_taz = {5010:15, 5002:10}
reduce_taz = {5002:10}
reduce_taz = {}
trip_dict = {key:[] for key, val in reduce_taz.items()}
print(f"initial number of trips: {len(root.findall('trip'))}")

initial number of trips: 166597


In [3]:
for trip in (root.findall('trip')):
    otaz = int(trip.get('fromTaz'))
    if otaz in reduce_taz.keys():
        trip_dict[otaz].append(trip)

In [4]:
for taz in trip_dict.keys():
    reduction_percent = reduce_taz[taz]
    print(f'taz: {taz}, reduction percentage: {reduction_percent}')
    reduction_number = int(reduction_percent/100 * len(trip_dict[taz]))
    for i in tqdm(range(reduction_number)):
        idx = random.randint(0, len(trip_dict[taz])-1)
        trip_to_remove = trip_dict[taz][idx]
        root.remove(trip_to_remove)
        trip_dict[taz].remove(trip_to_remove)


### moving from 5010 to 5000
for trips going to sumo_North TAZs: 

remove 30 percent and move another 15 percent to I5N

In [5]:
sr99N_sumoN = []
tobe_removed = 25
tobe_moved = 15

with open('sumo_south.txt', 'r') as f:
        content = f.read()
        sumoS_tazs = eval(content)



In [6]:
counter5000=0
counter5010=0
for trip in (root.findall('trip')):
    otaz = (trip.get('fromTaz'))
    if otaz is not None:
        otaz = int(otaz)
        if otaz==5010:
            dtaz = (trip.get('toTaz'))
            if (dtaz is not None) and (dtaz not in sumoS_tazs):
                dtaz = int(dtaz)
                sr99N_sumoN.append(trip)
                counter5010+=1
        if otaz==5000:
            counter5000+=1
print(counter5000)
print(counter5010)

26248
15909


In [7]:
reduction_number = int(tobe_removed/100 * len(sr99N_sumoN))
move_number = int(tobe_moved/100 * len(sr99N_sumoN))

for i in tqdm(range(reduction_number)):
    idx = random.randint(0, len(sr99N_sumoN)-1)
    trip_to_remove = sr99N_sumoN[idx]
    root.remove(trip_to_remove)
    sr99N_sumoN.remove(trip_to_remove)

for i in tqdm(range(move_number)):
    idx = random.randint(0, len(sr99N_sumoN)-1)
    trip_to_move = sr99N_sumoN[idx]
    trip_to_move.set('fromTaz', str(5000))
    sr99N_sumoN.remove(trip_to_move)

100%|███████████████████████████████████████████████████████████████████████████| 2386/2386 [00:00<00:00, 10955.19it/s]


In [8]:
counter5000=0
counter5010=0
for trip in (root.findall('trip')):
    otaz = int(trip.get('fromTaz'))
    if otaz==5010:
        dtaz = int(trip.get('toTaz'))
        if dtaz not in sumoS_tazs:
            counter5010+=1
    if otaz==5000:
        counter5000+=1
print(counter5000)
print(counter5010)

28634
7777


In [10]:
print(f"final number of trips: {len(root.findall('trip'))}")
# tree.write('reduced3_new_psrc_vehicle_trips_converted_taz.rou.xml', encoding='utf-8', xml_declaration=True)
tree.write('reduced_od2trips_new_psrc_vehicle_trips_converted_taz.rou.xml', encoding='utf-8', xml_declaration=True)

final number of trips: 162620


---------
### misc

In [58]:
tree = ET.parse('clean corrected inputs/new_bus_link_route.rou.xml')
root = tree.getroot()
print(f"tripinfos: {len(root.findall('trip'))}")

tripinfos: 1853


In [6]:
demand_file = 'adjusted_alaskanway_od2trips_psrc_vehicle_trips_converted_taz.rou.xml'
output_folder = 'new output'
output_file = output_folder + '/tripinfo_output.xml'
vehroute_output = output_folder + '/vehroute_output.xml'

In [7]:
tree = ET.parse(demand_file)
root = tree.getroot()
print(f"tripinfos: {len(root.findall('trip'))}")

# tree = ET.parse(vehroute_output)
# root = tree.getroot()
# print(f"vehicles: {len(root.findall('vehicle'))}")

tripinfos: 166597


In [56]:
file_path = 'old simulation files/new output/tripinfo_output.xml'
target_phrase1 = 'tripinfo id='

c1 = 0
with open(file_path, 'r') as file:
    for line_number, line in enumerate(file, start=1):
        if target_phrase1 in line:
            c1+=1
print(c1)

159436


In [65]:
file_path = output_folder+ '/log.xml'
target_phrase1 = 'No route for vehicle'
target_phrase2 = 'Removing'

c1 = 0
c2 = 0 

with open(file_path, 'r') as file:
    for line_number, line in enumerate(file, start=1):
        if target_phrase1 in line:
            c1+=1
            print(f"Line {line_number}: {line.strip()}")
        if target_phrase2 in line:
            c2+=1
            print(f"Line {line_number}: {line.strip()}")
print(f'no routes: {c1}')
print(f'removes: {c2}')

Line 860: Warning: No route for vehicle '1705' found.
Line 3082: Warning: No route for vehicle '11244' found.
Line 3462: Warning: No route for vehicle '7219' found.
Line 3467: Warning: No route for vehicle '9599' found.
Line 3468: Warning: Removing vehicle '9599' which has no valid route.
Line 4198: Warning: No route for vehicle '11558' found.
Line 4199: Warning: Removing vehicle '11558' which has no valid route.
Line 4592: Warning: No route for vehicle '8781' found.
Line 4598: Warning: No route for vehicle '16417' found.
Line 4599: Warning: Removing vehicle '16417' which has no valid route.
Line 6290: Warning: No route for vehicle '18290' found.
Line 6291: Warning: Removing vehicle '18290' which has no valid route.
Line 6295: Warning: No route for vehicle '16013' found.
Line 6296: Warning: Removing vehicle '16013' which has no valid route.
Line 7047: Warning: No route for vehicle '22726' found.
Line 8310: Warning: No route for vehicle '20440' found.
Line 8312: Warning: No route for ve

In [68]:
demand_id = []
trip_id = []

tree = ET.parse(demand_file)
root = tree.getroot()
print(f"demand: {len(root.findall('trip'))}")
for trip in root.findall('trip'):
    i = trip.get('id')
    demand_id.append(i)

tree = ET.parse(output_file)
root = tree.getroot()
print(f"tripinfo: {len(root.findall('tripinfo'))}")
for trip in root.findall('tripinfo'):
    i = trip.get('id')
    trip_id.append(i)
    
difference = list(set(demand_id)-set(trip_id))
print(len(difference))
print(difference)
# for i in demand_id:
#     if i not in trip_id:
#         print(i)

demand: 166597
tripinfo: 154213
14099
['158379', '141329', '164890', '155820', '151797', '160605', '148810', '162273', '141779', '149452', '163574', '164236', '149752', '141882', '143470', '159138', '150135', '158884', '159179', '137222', '138971', '151762', '163452', '142476', '156551', '150901', '140710', '137783', '149587', '161597', '138959', '151906', '158163', '152578', '153891', '142445', '145032', '138991', '160789', '164656', '138323', '153580', '154951', '156408', '158701', '151273', '144148', '147640', '150353', '137111', '155800', '144649', '138058', '141820', '165714', '154026', '145517', '141801', '151764', '151184', '139685', '158819', '161751', '158458', '140567', '144154', '152919', '145170', '143338', '150672', '153028', '140156', '148313', '163856', '159463', '153879', '148115', '152510', '162028', '137118', '141912', '156778', '165414', '163089', '153699', '152820', '138173', '159840', '141818', '162162', '154118', '147047', '150879', '139913', '157968', '159864', '

In [71]:
sorted(difference)

['104239',
 '104245',
 '108256',
 '11558',
 '120736',
 '125229',
 '128117',
 '137085',
 '137090',
 '137093',
 '137094',
 '137095',
 '137097',
 '137098',
 '137099',
 '137101',
 '137102',
 '137105',
 '137107',
 '137111',
 '137112',
 '137113',
 '137115',
 '137118',
 '137119',
 '137120',
 '137122',
 '137126',
 '137131',
 '137134',
 '137136',
 '137138',
 '137139',
 '137145',
 '137150',
 '137152',
 '137155',
 '137161',
 '137162',
 '137163',
 '137166',
 '137167',
 '137170',
 '137173',
 '137175',
 '137178',
 '137180',
 '137183',
 '137185',
 '137186',
 '137192',
 '137197',
 '137200',
 '137202',
 '137206',
 '137207',
 '137208',
 '137210',
 '137215',
 '137220',
 '137222',
 '137223',
 '137226',
 '137227',
 '137231',
 '137234',
 '137236',
 '137239',
 '137243',
 '137248',
 '137252',
 '137253',
 '137254',
 '137257',
 '137261',
 '137263',
 '137264',
 '137267',
 '137271',
 '137273',
 '137275',
 '137276',
 '137280',
 '137282',
 '137284',
 '137286',
 '137287',
 '137295',
 '137298',
 '137299',
 '137302',


# vtype calibrator

In [10]:
def get_calibrator(calibratorID, edgeID):
    line1 = f'\t<calibrator id="{calibratorID}" edge="{edgeID}" pos="0" type="passenger">'
    line2 = f'\t\t<flow begin="18000" end="36000" type="passengerHW"/>'
    return line1+'\n'+line2+'\n'+'\t</calibrator>'

In [11]:
with open('links_w_priority_over10.txt', 'r') as f:
    high_p_edges = f.readlines()
    high_p_edges = [line[5:-1] for line in high_p_edges]

all_lines = f'<additional>\n<vType id="passengerHW" vClass="passenger" accel="3.2" color="yellow" decel="3.5" length="4" maxSpeed="200" lcStrategic="1" lcSpeedGain="1" lcCooperative="1" lcSpeedGainLookahead="5" minGap="1.5" cc1="0.9" cc2="4" cc3="-8" cc4="-0.1" cc5="0.1" cc6="11.44" cc7="0.25" cc8="3.5" cc9="1.5"/>\n'

for i in range(len(high_p_edges)):
    all_lines = all_lines+get_calibrator(f'typeCalib{i}', high_p_edges[i])+'\n'
all_lines = all_lines + '</additional>'

with open('calibrators.add.xml', 'w') as f:
    f.write(all_lines)